In [5]:
import newspaper
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import string
from textblob import TextBlob
import json
import re
import glob
import pickle
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary

In [2]:
buzzfeed_paper = newspaper.build('http://www.buzzfeed.com/news', memoize_articles=False)

In [3]:
buzzfeed_paper.size()

2923

In [8]:
article_list = []
for article in buzzfeed_paper.articles:
    art = {}
    article.download()
    article.parse()
    text = process_text(article.text)
    title = clean_text(article.title)
    m = re.match(r'\A[0-9]+', title)
    if len(text) > 5 or not m:
        art["title"] = title
        art["authors"] = article.authors
        art["text"] = text
        art["sentiment"] = objectivity_sentiment(text)
        article_list.append(art)

with open('buzzfeed_articles2.json', 'w') as fp:
    json.dump(article_list, fp, indent=4, sort_keys=True)

CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
CRITICAL:newspaper.article:jpeg error wi

In [ ]:
with open('fox_articles.json', 'w') as fp:
    json.dump(article_list, fp, indent=4, sort_keys=True)

In [ ]:
sents_clean = []
for s in sents:
    removed = s.replace("\n\n", " ")
    clean = filter(lambda x: x in string.printable, removed)
    sents_clean.append("".join(l for l in clean if l not in string.punctuation.replace(".","")))
print sents_clean

In [ ]:
sentences = []
for s in sents_clean:
    sentences.append(sent_tokenize(s))
print sentences

In [ ]:
text = []
for s in sentences:
    text.append(s.replace(".",""))
print text

In [ ]:
final_text = []
for t in text:
    final_text.append(word_tokenize(t))
print final_text

In [ ]:
sentiments = []
for document in sentences:
    doc_sentiments = []
    for sent in document:
        doc_sentiments.append(TextBlob(sent).sentiment[1])
    sentiments.append(doc_sentiments)
print sentiments

In [ ]:
avg_sent = []
for s in sentiments:
    if len(s) != 0:
        avg_sent.append(sum(s) / float(len(s)))
print avg_sent

In [ ]:
len(avg_sent)

In [ ]:
total_sent = sum(avg_sent) / float(len(avg_sent))
print total_sent

In [2]:
def clean_text(text):
    removed = text.replace("\n\n", " ")
    clean = filter(lambda x: x in string.printable, removed)
    return "".join(l for l in clean if l not in string.punctuation.replace(".",""))

def tokenize_text(text):
    sents = sent_tokenize(text)
    return [sent for sent in sents if len(sent.split()) >= 5]

def objectivity_sentiment(text):
    sentiments = []
    for sent in text:
        sentiments.append(TextBlob(sent).sentiment[1])
    if len(sentiments) == 0:
        return []
    else:
        return sum(sentiments) / float(len(sentiments))

def process_text(text):
    clean = clean_text(text)
    tokens = tokenize_text(clean)
    return tokens


In [ ]:
def get_articles(name, url):
    paper = newspaper.build(url, memoize_articles=False)
    print name, url, paper.size()
    article_list = []
    for article in paper.articles:
        art = {}
        article.download()
        article.parse()
        text = process_text(article.text)
        if text:
            art["title"] = article.title
            art["authors"] = article.authors
            art["text"] = text
            art["sentiment"] = objectivity_sentiment(text)
            article_list.append(art)
            
    filename = name + '_articles.json'
    with open(filename, 'w') as fp:
        json.dump(article_list, fp, indent=4, sort_keys=True)

In [ ]:
news_df = pd.read_csv("news_sources.csv")
news_df.columns

In [ ]:
names = news_df['outlet'].tolist()
urls = news_df['url'].tolist()
for n, u in zip(names,urls):
    get_articles(n,u)

In [18]:
files = glob.glob('temp/*.json')
print len(files)

22


In [7]:
for fname in files:
    with open(fname) as data_file:    
        data = json.load(data_file)
        d_list = filter_short(data)
    with open(fname, 'w') as fp:
        json.dump(d_list, fp, indent=4, sort_keys=True)

In [6]:
def filter_short(data):
    d_list = []
    for d in data:
        if len(d["text"]) > 10:
            d_list.append(d)
    return d_list

In [13]:
def sum_sentiments(data):
    sent = 0
    for d in data:
        sent += d["sentiment"]
    return sent/len(data)

In [75]:
sent_list = []
for fname in files:
    with open(fname) as data_file:    
        data = json.load(data_file)
        sent = sum_sentiments(data)
        sent_list.append((fname, sent))
sents = sorted(sent_list, key=lambda x: x[1])
ranking = ""
for i, x in enumerate(sents):
    ranking += str(i+1)+', ' + x[0] + '\n'
print ranking

1, temp/RT_articles.json
2, temp/Reuters_articles.json
3, temp/CBSNews_articles.json
4, temp/WashingtonPost_articles.json
5, temp/ABC_articles.json
6, temp/Bloomberg_articles.json
7, temp/fox_articles.json
8, temp/NBC_articles.json
9, temp/USAToday_articles.json
10, temp/CNN_articles.json
11, temp/Economist_articles.json
12, temp/NYPost_articles.json
13, temp/BBC_articles.json
14, temp/Guardian_articles.json
15, temp/AlJazeera_articles.json
16, temp/NPR_articles.json
17, temp/NYT_articles.json
18, temp/ArsTechnica_articles.json
19, temp/Buzzfeed_articles.json
20, temp/HuffPost_articles.json
21, temp/Wired_articles.json
22, temp/Verge_articles.json



In [13]:
fname = "temp/CNN_articles.json"

In [14]:
 with open(fname) as data_file:    
        data = json.load(data_file)

In [17]:
article_list = []
for art in data:
    text = []
    for sent in art["text"]:
        sent = re.sub("Hide Caption [0-9]+ of [0-9]+", "", sent)
        text.append(sent)
    art["text"] = text
    article_list.append(art)
        
with open("temp/CNN_articles2.json", 'w') as fp:
        json.dump(article_list, fp, indent=4, sort_keys=True)

In [22]:
print len(article_list)

847


In [33]:
def filter_sentences(text):
    sentences = []
    for sent in text:
        spl = sent.split()
        if len(spl) > 5:
            sentences.append(sent)
    return sentences

In [37]:
def filter_text(article_list):    
    a_list = []
    for art in article_list:
        text = art["text"]
        sents = filter_sentences(text)
        if len(sents) > 15:
            art["text"] = sents
            art["sentiment"] = objectivity_sentiment(sents)
            a_list.append(art)
    return a_list

In [39]:
for fname in files:
    with open(fname) as data_file:    
        data = json.load(data_file)
        d_list = filter_text(data)
    with open('temp/' + fname, 'w') as fp:
        json.dump(d_list, fp, indent=4, sort_keys=True)

In [19]:
corpus = []
for fname in files:
    with open(fname) as data_file:    
        data = json.load(data_file)
        for d in data:
            corpus.append(' '.join(d['text']))
            
output = open('corpus.pkl', 'wb')
pickle.dump(corpus, output)
output.close()

In [20]:
pkl_file = open('corpus.pkl', 'rb')
corpus = pickle.load(pkl_file)
print len(corpus)
pkl_file.close()

5036


In [5]:
sents_sentiments = []
for s in corpus:
    sents_sentiments.append((s, TextBlob(s).sentiment[1]))
print sorted(sents_sentiments, key=lambda x: x[1], reverse=True)[:100]

[(u'Transcript for GMA Deals and Steals Oprah Winfrey Edition Tell us about this  oh nice.', 1.0), (u'When Oprah doesnt make her own sauce this is her own go to Susan hand made no artificial flavoring.', 1.0), (u'As the September meeting of the Fed board approached this rateanoia intensified only to subside a bit after the Fed decided not to raise rates.', 1.0), (u'The only problem with all this is that it has no basis in reality.', 1.0), (u'What I can be the final phase of this project which is opening a permanent rest and thats hammer on them.', 1.0), (u'She was really cute Cheryl Hart told 2020.', 1.0), (u'Sometimes the girl acted out violently.', 1.0), (u'And the Harts said they werent the only ones who opposed the adoption.', 1.0), (u'Excerpted from Killing Reagan The Violent Assault That Changed a Presidency.', 1.0), (u'His eyes scan the vast outdoor Los Angeles Coliseum with its vaulting peristyle arches and Olympic cauldron signifying the Olympic Games held there in 1932.', 1.0

In [9]:
print corpus[0]

Transcript for GMA Deals and Steals Oprah Winfrey Edition Tell us about this  oh nice. The fashionista Barbie the new one allows all girls the ability to see themselves in their dolls. You know why Oprah loved this. Growing up she could never find a doll that looked like herself. Now Barbie has all skin tone all body type all hair colors and hairstyles. So normally 20 for two of them. Youre going to select any two from the 18 choices 10 plus free shipping. Shes all about fitness Oprah is. All about fitness the best fitness tracker out there. Itting tras your sleep and monitors all activity and heart rate. Never been offered at a student like this. Today only for Gma viewers slashed half 90 Thats it Okay. You have got to feel these night shirts robin. Feel how soft and cozy and great these are. I wish you looked at me the way you look at your phone. This is what made Oprah crack up too. Generousry cut and made in the usa. Also available in plus size as well. Normally each of them 68 tod